# Azure OpenAI Chat Agents with Streaming

## Overview

This notebook demonstrates streaming workflows using **Azure OpenAI Chat Agents**. Similar to Azure AI Agents but using the Azure Open AI Service directly.

### Pipeline:
```
Writer Agent → Reviewer Agent (streaming)
```

### Key Concepts:
- Azure OpenAI Service integration
- Real-time token streaming via `AgentRunUpdateEvent`
- `add_agent()` with `output_response=True`
- Adaptive workflow modes (streaming vs non-streaming)

### Prerequisites:
- ✅ Azure OpenAI Service configured
- ✅ Environment variables: `AZURE_OPENAI_ENDPOINT`, `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- ✅ Azure CLI authentication (`az login`)

In [1]:
import asyncio

from agent_framework import AgentRunUpdateEvent, WorkflowBuilder, WorkflowOutputEvent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../../.env')


True

In [2]:
# Create Azure OpenAI chat client
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
chat_client = AzureOpenAIChatClient(
    deployment_name=deployment_name,
    endpoint=endpoint,
    credential=AzureCliCredential()
)
print("✅ Azure OpenAI Chat Client created")

✅ Azure OpenAI Chat Client created


In [3]:
# Create Writer Agent
writer_agent = chat_client.create_agent(
    instructions=(
        "You are an excellent content writer. You create new content and edit contents based on the feedback."
    ),
    name="writer_agent",
)
print("✅ Writer Agent created")

✅ Writer Agent created


In [4]:
# Create Reviewer Agent
reviewer_agent = chat_client.create_agent(
    instructions=(
        "You are an excellent content reviewer."
        "Provide actionable feedback to the writer about the provided content."
        "Provide the feedback in the most concise manner possible."
    ),
    name="reviewer_agent",
)
print("✅ Reviewer Agent created")

✅ Reviewer Agent created


In [5]:
# Build workflow
workflow = (
    WorkflowBuilder()
    .add_agent(writer_agent, id="Writer")
    .add_agent(reviewer_agent, id="Reviewer", output_response=True)
    .set_start_executor(writer_agent)
    .add_edge(writer_agent, reviewer_agent)
    .build()
)
print("✅ Workflow built: Writer → Reviewer")

AttributeError: 'WorkflowBuilder' object has no attribute 'add_agent'

In [ ]:
# Run with streaming
user_message = "Create a slogan for a new electric SUV that is affordable and fun to drive."
print(f"\n📝 User: {user_message}\n" + "=" * 70 + "\n")

last_executor_id: str | None = None

events = workflow.run_stream(user_message)
async for event in events:
    if isinstance(event, AgentRunUpdateEvent):
        eid = event.executor_id
        if eid != last_executor_id:
            if last_executor_id is not None:
                print()
            print(f"\n🤖 {eid}:", end=" ", flush=True)
            last_executor_id = eid
        print(event.data, end="", flush=True)
    elif isinstance(event, WorkflowOutputEvent):
        print("\n\n" + "=" * 70)
        print("\n📤 Final Output:\n")
        print(event.data)

print("\n" + "=" * 70 + "\n✅ Complete!")

## Key Differences: Azure OpenAI vs Azure AI Agents

| Feature | Azure OpenAI | Azure AI Agents |
|---------|--------------|----------------|
| **Service** | Azure OpenAI Service | Azure AI Agent Service |
| **Client** | `AzureOpenAIChatClient` | `AzureAIAgentClient` |
| **Setup** | Direct instantiation | Async context managers |
| **Auth** | API key or credential | Credential required |
| **Models** | OpenAI models | Various AI models |

Both support streaming and integrate seamlessly with workflows!